In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import sys

In [2]:
sns.set_theme(style="whitegrid")

In [3]:
import ast

In [4]:
PREFIX_PATH = 'data'
df = pd.read_csv(f'{PREFIX_PATH}/final_data_complete.csv')
df = df[['WikiID', 'merge_year', 'Genres', 'distilbert_emotions', 'ActorAge', 'ActorGender', 'ActorBirthDate', 'ActorGenderFlag']].copy()

df = df.dropna(subset=['distilbert_emotions'])
print("Number of rows", len(df))
df['distilbert_emotions'] = df['distilbert_emotions'].apply(lambda x: {} if pd.isna(x) else ast.literal_eval(x))
df.head()

Number of rows 33684


,WikiID,merge_year,Genres,distilbert_emotions,ActorAge,ActorGender,ActorBirthDate,ActorGenderFlag
0,975900,2001.0,"['Thriller', 'Science Fiction', 'Horror', 'Adv...","{'disgust': 0.5021221041679382, 'fear': 0.3510...","[42.0, 27.0, 32.0, 33.0, 23.0, 52.0, 56.0, 37....","['F', 'F', 'M', 'M', 'F', 'F', 'F', 'M', 'M', ...","[Timestamp('1958-08-26 00:00:00'), Timestamp('...","[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,9363483,1987.0,"['Thriller', 'Erotic thriller', 'Psychological...","{'anger': 0.5024036765098572, 'disgust': 0.324...","[32.0, 26.0]","['M', 'F']","[Timestamp('1954-05-08 00:00:00'), Timestamp('...","[0, 1]"
4,261236,1983.0,['Drama'],"{'anger': 0.3256497378461063, 'disgust': 0.163...","[32.0, 32.0, 35.0]","['F', 'M', 'M']","[Timestamp('1950-06-20 00:00:00'), Timestamp('...","[1, 0, 0]"
12,6631279,1997.0,"['Romantic comedy', 'Ensemble Film', 'Comedy-d...","{'sadness': 0.9155038595199585, 'neutral': 0.0...","[25.0, 33.0, 37.0, 35.0, 33.0, 48.0]","['M', 'F', 'F', 'M', 'F', 'F']","[Timestamp('1971-09-15 00:00:00'), Timestamp('...","[0, 1, 1, 0, 1, 1]"
13,171005,1989.0,"['Costume drama', 'War film', 'Epic', 'Period ...","{'neutral': 0.6794102787971497, 'surprise': 0....","[28.0, 51.0, 33.0, 26.0, 30.0, 32.0, 58.0, 53....","['M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', ...","[Timestamp('1960-12-10 00:00:00'), Timestamp('...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ..."


In [5]:
emotions = ['disgust', 'fear', 'anger', 'neutral', 'sadness', 'surprise', 'joy']
for emotion in emotions:
    df[emotion] = df['distilbert_emotions'].apply(lambda x: x[emotion])
df.head()

,WikiID,merge_year,Genres,distilbert_emotions,ActorAge,ActorGender,ActorBirthDate,ActorGenderFlag,disgust,fear,anger,neutral,sadness,surprise,joy
0,975900,2001.0,"['Thriller', 'Science Fiction', 'Horror', 'Adv...","{'disgust': 0.5021221041679382, 'fear': 0.3510...","[42.0, 27.0, 32.0, 33.0, 23.0, 52.0, 56.0, 37....","['F', 'F', 'M', 'M', 'F', 'F', 'F', 'M', 'M', ...","[Timestamp('1958-08-26 00:00:00'), Timestamp('...","[1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.502122,0.351022,0.065147,0.057085,0.018771,0.004086,0.001766
3,9363483,1987.0,"['Thriller', 'Erotic thriller', 'Psychological...","{'anger': 0.5024036765098572, 'disgust': 0.324...","[32.0, 26.0]","['M', 'F']","[Timestamp('1954-05-08 00:00:00'), Timestamp('...","[0, 1]",0.324022,0.032032,0.502404,0.096385,0.037811,0.005438,0.001907
4,261236,1983.0,['Drama'],"{'anger': 0.3256497378461063, 'disgust': 0.163...","[32.0, 32.0, 35.0]","['F', 'M', 'M']","[Timestamp('1950-06-20 00:00:00'), Timestamp('...","[1, 0, 0]",0.163747,0.002405,0.325650,0.440068,0.025255,0.005851,0.037024
12,6631279,1997.0,"['Romantic comedy', 'Ensemble Film', 'Comedy-d...","{'sadness': 0.9155038595199585, 'neutral': 0.0...","[25.0, 33.0, 37.0, 35.0, 33.0, 48.0]","['M', 'F', 'F', 'M', 'F', 'F']","[Timestamp('1971-09-15 00:00:00'), Timestamp('...","[0, 1, 1, 0, 1, 1]",0.021709,0.010488,0.006082,0.042634,0.915504,0.002163,0.001421
13,171005,1989.0,"['Costume drama', 'War film', 'Epic', 'Period ...","{'neutral': 0.6794102787971497, 'surprise': 0....","[28.0, 51.0, 33.0, 26.0, 30.0, 32.0, 58.0, 53....","['M', 'M', 'M', 'M', 'F', 'M', 'M', 'M', 'M', ...","[Timestamp('1960-12-10 00:00:00'), Timestamp('...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, ...",0.004746,0.003841,0.016094,0.679410,0.057925,0.129912,0.108072


In [8]:
import plotly.express as px

emotion_sums = df[['disgust', 'fear', 'anger', 'neutral', 'sadness', 'surprise', 'joy']].sum()
fig = px.pie(
    names=emotion_sums.index, 
    values=emotion_sums.values, 
    title="Emotion Distribution Across All Movies"
)
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',  # Transparent background for the paper
    plot_bgcolor='rgba(0,0,0,0)',   # Transparent background for the plot
    title_font=dict(color="white"), # White color for the title
    legend_font=dict(color="white") # White color for the legend text
)

# Save the chart as a PNG image
fig.write_image("emotion_distribution_pie_chart_no_bg.png")

print("Pie chart saved as 'emotion_distribution_pie_chart_no_bg.png'")

fig.show()

Pie chart saved as 'emotion_distribution_pie_chart_no_bg.png'
